# Final Year Project - Intelligent Health Monitoring System
#### by: Koo Chia Meng, A0165275Y

### Creating Object Class for subjects
Since the data would be gathered from a few people, it would be neater to put them as objects in Object Oriented Programming (OOP). 

The below code cells are copied from the Data Acqusition, Cleaning Jupyter Notebook. The purpose is to allow importing of class into other Notebooks.

### Definining Functions to be declared as methods

In [1]:
# Creating Functions to be used for the script

def Fitbit_Authorise(client_id,client_secret,redirect_uri):
    
    Fitbit_Auth_URL = 'https://www.fitbit.com/oauth2/authorize'
    Fitbit_token_URL = 'https://api.fitbit.com/oauth2/token'
    
    # Initialize client
    scope = ["activity", "heartrate", "profile", "sleep", "weight"] # Specify required data authorisation
    oauth = requests_oauthlib.OAuth2Session(client_id, redirect_uri=redirect_uri, scope=scope)

    # Fetching URL for the relevant Autourisation page so that users can log in with browser.
    authorization_url, state = oauth.authorization_url(Fitbit_Auth_URL)
    
    webbrowser.open_new(authorization_url) #direct user to log in
        
    print('Broweser will call this URL (send this to the subject): '+ authorization_url)
    
    Auth_response = input(' Copy & Paste the full callback URL:') # Ask user to input the return URL

    token = oauth.fetch_token(Fitbit_token_URL, client_secret=client_secret,authorization_response='https://'+Auth_response)

    return (token)

def Withings_Authorise(client_id,client_secret,redirect_uri):

    Withings_Auth_URL = 'https://account.withings.com/oauth2_user/authorize2'
    Withings_token_URL = 'https://account.withings.com/oauth2/token'
    
    scope_wt = ["user.activity"] # Specify required data authorisation
    oauth_wt = requests_oauthlib.OAuth2Session(client_id, redirect_uri=redirect_uri, scope=scope_wt)
    
    # Fetching URL for the relevant Autourisation page so that users can log in with browser.
    authorization_url_wt, state = oauth_wt.authorization_url(Withings_Auth_URL)
    
    webbrowser.open_new(authorization_url_wt) #direct user to log in to grant permission
    
    Auth_response_wt = input(' Copy & Paste the full callback URL:') # Ask user to input the return URL

# !!! NOT WORKING !!!
#     token_wt = oauth_wt.fetch_token(Withings_token_URL,client_id=client_id,client_secret=client_secret,authorization_response='https://'+Auth_response_wt)
    
    # Have to use the Manual post method defined above
    token_wt = fetch_withings_token(Auth_response_wt,Withings_token_URL,client_id,client_secret,redirect_uri)
    
    return (token_wt)


# ONLY USE WHEN REQUIRED!!!

# REFRESH TOKEN
def Refresh_Token_Fitbit(token):
    HEADERS = {'Authorization':'Bearer '+token['access_token'], 
               'Content-Type':'application/x-www-form-urlencoded'} 
    PARAMS = {'grant_type':'refresh_token','refresh_token':token['refresh_token']}
    Fitbit_Refresh_Token_API = 'https://api.fitbit.com/oauth2/token'
    
    # sending get request and saving the response as response object 
    r = requests.post(Fitbit_Refresh_Token_API,headers=HEADERS,data=PARAMS) 
    return r.json()

### [ FITBIT ] Fetching Data From  API:

##### Profiles [Note! The details in the profile are input by the user, might not be correct]:

In [2]:
#Fetch Profile details

def Fetch_Profile_Fitbit(input_token):
    HEADERS = {'Authorization':'Bearer '+ input_token['access_token']} 
    Fitbit_Profile_API = 'https://api.fitbit.com/1/user/'+ input_token['user_id']+'/profile.json'
    # sending get request and saving the response as response object 
    r = requests.get(Fitbit_Profile_API,headers=HEADERS) 
    
    extract = {}
    data = r.json()['user']
    
    # extracting data in JSON format 
    detail = ['age','gender','height','weight']
    
    for i in detail:
        extract[i] = data[i]
        
    return extract

##### Heart Rate per day:

In [3]:
# Fetching HR Data from FITBIT

def Fetch_HR_Fitbit(date1,date2,token):
    HEADERS = {'Authorization':'Bearer '+ token['access_token']} 
    Fitbit_HR_API = 'https://api.fitbit.com/1/user/'+token['user_id']+'/activities/heart/date/'+date1+'/'+date2+'.json'

    # sending get request and saving the response as response object 
    r = requests.get(Fitbit_HR_API,headers=HEADERS) 

    # extracting data in JSON format 
    return r.json()['activities-heart']


##### Intra - Day Heart Rate: (THE API ONLY WORKS IF YOU FETCH 1 DAY AT A TIME!!!)

In [4]:
# Fetching INTRADAY HR Data from FITBIT (By per Minute '1min' or per Second '1sec')
# NOTE THE DATE DATE RANGE TO PREVENT LONG RUNNING SCRIPT!!!

def Fetch_Intra_Day_HR_Fibit(date1,date2,token):
    
    start_date = datetime.datetime.strptime(date1,"%Y-%m-%d") 
    end_date = datetime.datetime.strptime(date2,"%Y-%m-%d")
    
    date_range = pd.date_range(start=start_date,end=end_date).to_pydatetime().tolist()
    str_date_range = [datetime.datetime.strftime(x,"%Y-%m-%d") for x in date_range]

    HEADERS = {'Authorization':'Bearer '+ token['access_token']} 
    # Fitbit_Intra_HR_API = 'https://api.fitbit.com/1/user/'+token['user_id']+'/activities/heart/date/'+date1+'/1d/1min.json'

    data_Intra_HR_fitbit_Dict = {}

    for d in str_date_range:

        Fitbit_Intra_HR_API = 'https://api.fitbit.com/1/user/'+ token['user_id']+'/activities/heart/date/'+str(d)+'/1d/1sec.json'

        # sending get request and saving the response as response object 
        r = requests.get(Fitbit_Intra_HR_API,headers=HEADERS) 

        # extracting data in JSON format 
        data_Intra_HR_fitbit_Dict[d] = r.json()['activities-heart-intraday']
    
    return data_Intra_HR_fitbit_Dict

##### Sleep per day:

In [5]:
# Fetching Sleep Data from FITBIT

def Fetch_Sleep_Fitbit(date1,date2,token):
    HEADERS = {'Authorization':'Bearer '+ token['access_token']} 
    Fitbit_Sleep_API = 'https://api.fitbit.com/1.2/user/'+ token['user_id']+'/sleep/date/'+date1+'/'+date2+'.json'

    # sending get request and saving the response as response object 
    r = requests.get(Fitbit_Sleep_API,headers=HEADERS) 

    # extracting data in JSON format 
    return r.json()['sleep']


##### Activity (Steps) per day:

In [6]:
# Fetching Steps Data from FITBIT

def Fetch_Step_Activity_Fitbit(date1,date2,token):
    HEADERS = {'Authorization':'Bearer '+ token['access_token']} 
    Fitbit_Activity_Steps_API = 'https://api.fitbit.com/1/user/'+ token['user_id']+'/activities/tracker/steps/date/'+date1+'/'+date2+'.json'

    # sending get request and saving the response as response object 
    r = requests.get(Fitbit_Activity_Steps_API,headers=HEADERS) 

    # extracting data in JSON format 
    return r.json()['activities-tracker-steps']


### SUBJECT CLASS

In [7]:
# Creating class for test subjects, since the data collected would be from several people. 

class DATA():
    def __init__(self):
        self.profile = None        
        self.heart_data = None
        self.intra_heart_data = None
        self.sleep_data = None
        self.step_activity_data = None

class SUBJECT(DATA):
    def __init__(self,tracker):
        self.tracker = tracker
        self.last_fetch = None
        self.token = None
    
# Related Methods: Authorise, fetch datas. these methods will cater for both fitbit and withings.
    def Authorise(self,client_id,client_secret,redirect_uri):
        error_exist = True
        if self.tracker == 'fitbit':
            try:
                self.token = Fitbit_Authorise(client_id,client_secret,redirect_uri)
                error_exist = False
            except Exception as e:
                error_exist = True
                print (type(e),str(e))
        elif self.tracker == 'withings':
            try:
                self.token = Withings_Authorise(client_id,client_secret,redirect_uri)
                error_exist = False
            except Exception as e:
                error_exist = True
                print (type(e), str(e))
        if error_exist: 
            print('Authourisation Failed. Please Check Inputs Again.')
        else: print('Success!')

    def Refresh_Token(self):    
        error_exist = True
        if self.tracker == 'fitbit':
            try:
                self.token = Refresh_Token_Fitbit(self.token)
                error_exist = False
            except Exception as e:
                error_exist = True
                print (type(e), str(e))
        elif self.tracker == 'withings':
            try:
                pass
                error_exist = False
            except Exception as e:
                error_exist = True
                print (type(e), str(e))
        if error_exist: 
            print('Authourisation Failed. Please Check Inputs Again.')
        else: print('Success!')

    def Fetch_Profile(self):                
        error_exist = True
        
        if self.tracker == 'fitbit':
            try:
                self.profile = Fetch_Profile_Fitbit(self.token)
                error_exist = False
            except:
                error_exist = True
        elif self.tracker == 'withings':
            try:
                pass
                error_exist = False
            except Exception as e:
                error_exist = True
        if error_exist: 
            print('Process Failed. Please Check Inputs or Authorisation Again.')
        else: print('Success!')
           
    def Fetch_HR_Data(self,date1,date2):                
        error_exist = True
        
        if self.tracker == 'fitbit':
            try:
                self.heart_data = Fetch_HR_Fitbit(date1,date2,self.token)
                error_exist = False
            except Exception as e:
                error_exist = True
                print (type(e), str(e))
        elif self.tracker == 'withings':
            try:
                pass
                error_exist = False
            except Exception as e:
                error_exist = True
                print (type(e), str(e))
        if error_exist: 
            print('Process Failed. Please Check Inputs or Authorisation Again.')
        else: print('Success!')
            
    def Fetch_IntraDayHR_Data(self,date1,date2):                
        error_exist = True
        
        if self.tracker == 'fitbit':
            try:
                self.intra_heart_data = Fetch_Intra_Day_HR_Fibit(date1,date2,self.token)
                error_exist = False
            except Exception as e:
                error_exist = True
                print (type(e), str(e))
        elif self.tracker == 'withings':
            try:
                pass
                error_exist = False
            except Exception as e:
                error_exist = True
                print (type(e), str(e))
        if error_exist: 
            print('Process Failed. Please Check Inputs or Authorisation Again.')
        else: print('Success!')
            
    def Fetch_Sleep_Data(self,date1,date2):                
        error_exist = True
        
        if self.tracker == 'fitbit':
            try:
                self.sleep_data = Fetch_Sleep_Fitbit(date1,date2,self.token)
                error_exist = False
            except Exception as e:
                error_exist = True
                print (type(e),str(e))
        elif self.tracker == 'withings':
            try:
                pass
                error_exist = False
            except Exception as e:
                error_exist = True
                print (type(e),str(e))
        if error_exist: 
            print('Process Failed. Please Check Inputs or Authorisation Again. \nThe time frame might be too long to fetch, consider cutting it to one month.')
        else: print('Success!')

    def Fetch_Step_Activity_Data(self,date1,date2):                
        error_exist = True
        
        if self.tracker == 'fitbit':
            try:
                self.step_activity_data = Fetch_Step_Activity_Fitbit(date1,date2,self.token)
                error_exist = False
            except Exception as e:
                error_exist = True
                print (type(e),str(e))
        elif self.tracker == 'withings':
            try:
                pass
                error_exist = False
            except Exception as e:
                error_exist = True
                print (type(e),str(e))
        if error_exist: 
            print('Process Failed. Please Check Inputs or Authorisation Again.')
        else: print('Success!')
            

### [ WITHINGS ] Fetching API Data:

In [8]:
# #Specifying Client ID and Client Secret from Withings
# CLIENT_ID_WIT = 'd38f63b6f9f8e1825e646c849fc8f2c62667d781cf86de79a20168ca5bca04c1'
# CLIENT_SECRET_WIT = 'a272d1bba681026fec24023fdbd56698e9161dbbe031bd05998d3a1e0dc61cb4'
# CALL_BACK_URL = 'http://127.0.0.1:8080/'

# token_wit = Withings_Authorise(CLIENT_ID_WIT,CLIENT_SECRET_WIT,CALL_BACK_URL)

# print (token_wit)

In [9]:
# date1 = "2020-02-19"
# date2 = str(datetime.datetime.now().strftime("%Y-%m-%d"))

# HEADERS = {'Authorization':'Bearer '+token_wit['access_token']} 
# Withings_Activity_API = 'https://wbsapi.withings.net/v2/measure?action=getactivity'

# PARAMS = {'startdateymd': date1,
#           'enddateymd' : date2
#          }

# # sending get request and saving the response as response object 
# r = requests.get(Withings_Activity_API,headers=HEADERS,params=PARAMS)

# # extracting data in JSON format 
# data_Withings = r.json()
# print (data_Withings)

### OTHER FITNESS TRACKER API